In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split


# Paths

BASE_DIR = "/kaggle/input/datasets/allandclive/chicken-disease-1"
TRAIN_CSV = os.path.join(BASE_DIR, "train_data.csv")


# Load Dataset

df = pd.read_csv(TRAIN_CSV)

print("Data loaded successfully")
print("Total images:", len(df))
print("Classes:", df['label'].unique())

image_column = [col for col in df.columns if col != 'label'][0]

df['filepath'] = df[image_column].apply(
    lambda x: os.path.join(BASE_DIR, "Train", x)
)

# Split dataset
train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df['label']
)

print("Training samples:", len(train_df))
print("Validation samples:", len(val_df))

# Parameters

IMG_SIZE = 224
BATCH_SIZE = 32

# Data Augmentation

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
    rescale=1./255
)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="filepath",
    y_col="label",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

validation_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col="filepath",
    y_col="label",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

NUM_CLASSES = len(train_generator.class_indices)

print("Number of classes:", NUM_CLASSES)

# Load ResNet50 


print("Loading ResNet50 model (offline safe)...")

base_model = ResNet50(
    weights=None,
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

base_model.trainable = False


# Classification Head

inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)

x = layers.BatchNormalization()(x)

x = layers.Dense(256, activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.4)(x)

x = layers.Dense(128, activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)

outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = keras.Model(inputs, outputs)


# Phase 1 Training
# Feature Extraction

print("Starting Phase 1 Training")

model.compile(
    optimizer=keras.optimizers.AdamW(
        learning_rate=1e-4,
        weight_decay=1e-5
    ),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.fit(
    train_generator,
    epochs=15,
    validation_data=validation_generator,
    callbacks=[
        EarlyStopping(
            monitor="val_loss",
            patience=5,
            restore_best_weights=True
        ),
        ModelCheckpoint(
            "best_model.h5",
            monitor="val_accuracy",
            save_best_only=True
        )
    ],
    verbose=1
)

# Phase 2 Fine Tuning


print("Starting Fine Tuning")

base_model.trainable = True

# Freeze BatchNorm layers
for layer in base_model.layers:
    if isinstance(layer, layers.BatchNormalization):
        layer.trainable = False

# Freeze early layers only
for layer in base_model.layers[:-40]:
    layer.trainable = False

model.compile(
    optimizer=keras.optimizers.AdamW(
        learning_rate=1e-5,
        weight_decay=1e-5
    ),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[
        EarlyStopping(
            monitor="val_loss",
            patience=3,
            restore_best_weights=True
        ),
        ModelCheckpoint(
            "best_model_finetuned.h5",
            monitor="val_accuracy",
            save_best_only=True
        )
    ],
    verbose=1
)

# Evaluation


model.load_weights("best_model_finetuned.h5")

val_loss, val_accuracy = model.evaluate(
    validation_generator,
    verbose=0
)

print("\nFinal Validation Accuracy:", round(val_accuracy, 4))
print("Training completed successfully.")

Data loaded successfully
Total images: 8067
Classes: ['Salmonella' 'Coccidiosis' 'New Castle Disease' 'Healthy']
Training samples: 6453
Validation samples: 1614
Found 6453 validated image filenames belonging to 4 classes.
Found 1614 validated image filenames belonging to 4 classes.
Number of classes: 4
Loading ResNet50 model (offline safe)...
Starting Phase 1 Training
Epoch 1/15
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5067 - loss: 1.4063

202/202 ━━━━━━━━━━━━━━━━━━━━ 481s 2s/step - accuracy: 0.5070 - loss: 1.4054 - val_accuracy: 0.2980 - val_loss: 1.5775
Epoch 2/15
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6318 - loss: 1.0081

202/202 ━━━━━━━━━━━━━━━━━━━━ 521s 3s/step - accuracy: 0.6318 - loss: 1.0080 - val_accuracy: 0.3092 - val_loss: 1.2788
Epoch 3/15
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6564 - loss: 0.9553

202/202 ━━━━━━━━━━━━━━━━━━━━ 471s 2s/step - accuracy: 0.6565 - loss: 0.9552 - val_accuracy: 0.7379 - val_loss: 0.8086
Epoch 4/15
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6766 - loss: 0.8894

202/202 ━━━━━━━━━━━━━━━━━━━━ 470s 2s/step - accuracy: 0.6766 - loss: 0.8894 - val_accuracy: 0.7491 - val_loss: 0.6746
Epoch 5/15
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6967 - loss: 0.8722

...

202/202 ━━━━━━━━━━━━━━━━━━━━ 465s 2s/step - accuracy: 0.7325 - loss: 0.7247 - val_accuracy: 0.7887 - val_loss: 0.5571
Epoch 15/15
202/202 ━━━━━━━━━━━━━━━━━━━━ 464s 2s/step - accuracy: 0.7466 - loss: 0.6911 - val_accuracy: 0.7869 - val_loss: 0.5617
Starting Fine Tuning
Epoch 1/10
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7316 - loss: 0.7475

202/202 ━━━━━━━━━━━━━━━━━━━━ 789s 4s/step - accuracy: 0.7316 - loss: 0.7475 - val_accuracy: 0.5359 - val_loss: 2.0490
Epoch 2/10
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7443 - loss: 0.7038

202/202 ━━━━━━━━━━━━━━━━━━━━ 732s 4s/step - accuracy: 0.7443 - loss: 0.7038 - val_accuracy: 0.5539 - val_loss: 2.3111
Epoch 3/10
202/202 ━━━━━━━━━━━━━━━━━━━━ 717s 4s/step - accuracy: 0.7421 - loss: 0.6921 - val_accuracy: 0.5229 - val_loss: 1.7369
Epoch 4/10
202/202 ━━━━━━━━━━━━━━━━━━━━ 763s 4s/step - accuracy: 0.7440 - loss: 0.6990 - val_accuracy: 0.5434 - val_loss: 2.7109
Epoch 5/10
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7550 - loss: 0.6760

202/202 ━━━━━━━━━━━━━━━━━━━━ 708s 4s/step - accuracy: 0.7551 - loss: 0.6759 - val_accuracy: 0.5967 - val_loss: 1.2661
Epoch 6/10
202/202 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7614 - loss: 0.6443

202/202 ━━━━━━━━━━━━━━━━━━━━ 705s 3s/step - accuracy: 0.7614 - loss: 0.6443 - val_accuracy: 0.6642 - val_loss: 1.2211
Epoch 7/10
202/202 ━━━━━━━━━━━━━━━━━━━━ 710s 4s/step - accuracy: 0.7741 - loss: 0.6410 - val_accuracy: 0.4876 - val_loss: 4.2819
Epoch 8/10
202/202 ━━━━━━━━━━━━━━━━━━━━ 712s 4s/step - accuracy: 0.7759 - loss: 0.6333 - val_accuracy: 0.5781 - val_loss: 1.2344
Epoch 9/10
202/202 ━━━━━━━━━━━━━━━━━━━━ 742s 4s/step - accuracy: 0.7764 - loss: 0.6019 - val_accuracy: 0.5713 - val_loss: 1.4026

Final Validation Accuracy: 0.6642
Training completed successfully.


In [6]:
train_df, temp_df = train_test_split(
    df,
    test_size=0.3,
    random_state=42,
    stratify=df['label']
)

val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    random_state=42,
    stratify=temp_df['label']
)

print("Train:", len(train_df))
print("Validation:", len(val_df))
print("Test:", len(test_df))

Train: 5646
Validation: 1210
Test: 1211


In [7]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col="filepath",
    y_col="label",
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    shuffle=False
)

Found 1211 validated image filenames belonging to 4 classes.


In [8]:
test_loss, test_accuracy = model.evaluate(test_generator, verbose=1)

print("Test Accuracy:", round(test_accuracy, 4))

38/38 ━━━━━━━━━━━━━━━━━━━━ 69s 2s/step - accuracy: 0.6581 - loss: 1.2499
Test Accuracy: 0.6722


In [9]:
predictions = model.predict(test_generator)

predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

38/38 ━━━━━━━━━━━━━━━━━━━━ 71s 2s/step


In [10]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(true_classes, predicted_classes, target_names=class_labels))

cm = confusion_matrix(true_classes, predicted_classes)
print(cm)

                    precision    recall  f1-score   support

       Coccidiosis       0.57      0.97      0.71       372
           Healthy       0.75      0.83      0.79       361
New Castle Disease       0.00      0.00      0.00        84
        Salmonella       0.88      0.40      0.55       394

          accuracy                           0.67      1211
         macro avg       0.55      0.55      0.51      1211
      weighted avg       0.68      0.67      0.63      1211

[[360   8   0   4]
 [ 54 298   0   9]
 [ 46  29   0   9]
 [175  63   0 156]]
